In [27]:
import os
import glob
import folium
import geopandas as gpd
from shapely.geometry import Point
import pandas as pd
import time
from IPython.display import clear_output

In [28]:
file_paths = [
    "./data/combined_train_schedules_rer_A.csv/",
    "./data/combined_train_schedules_rer_B.csv/",
    "./data/combined_train_schedules_rer_C.csv/",
    "./data/combined_train_schedules_rer_D.csv/",
    "./data/combined_train_schedules_rer_E.csv/",
]

# Define color mapping for RER lines
rer_colors = {
    'A': 'red',
    'B': 'blue',
    'C': 'goldenrod',
    'D': 'green',
    'E': 'purple'
}

In [29]:
file_patterns = [os.path.join(path, "part-*.csv") for path in file_paths]

# Use glob to get all part file paths
all_files = [file for pattern in file_patterns for file in glob.glob(pattern)]

# Initialize an empty list to hold DataFrames
dataframes = []

# Iterate over the file paths and read each CSV file
for file in all_files:
    df = pd.read_csv(file)
    dataframes.append(df)

# Concatenate all DataFrames into a single DataFrame
train_data = pd.concat(dataframes, ignore_index=True)

# Modify the journey_ids list to include the line information
journey_ids = [
    ("A", "vehicle_journey:SNCF:2024-07-08:BABY17:1187:RapidTransit"),
    ("B", "vehicle_journey:SNCF:2024-07-08:EACE07:1187:RapidTransit"),
    ("C", "vehicle_journey:SNCF:2024-07-08:141186-141187:1187:RapidTransit"),
    ("D", "vehicle_journey:SNCF:2024-07-08:122302-122303:1187:RapidTransit"),
    ("E", "vehicle_journey:SNCF:2024-07-08:116192:1187:RapidTransit"),
]

# Create a dictionary to hold GeoDataFrames for each journey
journeys = {}

for line, journey_id in journey_ids:
    journey = train_data[train_data["journey_id"] == journey_id]
    geometry = [Point(xy) for xy in zip(journey["longitude"], journey["latitude"])]
    journeys[line] = gpd.GeoDataFrame(journey, geometry=geometry)

In [30]:
# Fonction pour créer la carte folium
def creer_carte_folium(journeys, current_indices):
    # Créer une carte centrée sur Paris
    carte = folium.Map(location=[48.8566, 2.3522], zoom_start=11)

    for i, (line, gdf_journey) in enumerate(journeys.items()):
        current_index = current_indices[i]
        color = rer_colors[line]

        # Créer une liste de coordonnées pour la ligne complète
        locations = [(lat, lon) for lon, lat in zip(gdf_journey.geometry.x, gdf_journey.geometry.y)]
        folium.PolyLine(locations, color=color, weight=2, opacity=0.5, dash_array="5, 5", popup=f"RER {line} Path").add_to(carte)

        # Créer une liste de coordonnées pour la progression actuelle
        current_locations = locations[:current_index + 1]

        # Vérifier si current_locations a au moins deux coordonnées
        if len(current_locations) >= 2:
            folium.PolyLine(current_locations, color=color, weight=4, popup=f"RER {line} Progress").add_to(carte)

            # Add a marker for the current train position
            current_position = current_locations[-1]
            folium.CircleMarker(current_position, radius=5, color=color, fill=True, fill_color=color, popup=f"RER {line}").add_to(carte)

        # Add markers for the stations
        for idx, row in gdf_journey.iterrows():
            station_label = f"{line}{idx + 1}"
            folium.CircleMarker(
                [row.geometry.y, row.geometry.x],
                radius=3,
                color=color,
                fill=True,
                fill_color='white',
                popup=f"{station_label}: {row['stop_point_name']}"
            ).add_to(carte)

    return carte

In [35]:
# Simulate the journey progress
max_length = max(len(journey) for journey in journeys.values())
for i in range(max_length):
    current_indices = [min(i, len(journey)) - 1 for journey in journeys.values()]
    clear_output(wait=True)
    carte = creer_carte_folium(journeys, current_indices)  # Create the folium map
    display(carte)  # Display the map in the notebook
    time.sleep(1)